In [2]:
import sqlite3
from sqlite3 import Error
def create_connection(path):
    try:
        connection = sqlite3.connect(path)
        print('Connection to SQLite DB successful')
        return connection
    except Error as e:
        print(f'The error {e} occurred')
        return None


connection = create_connection("pizza_app.sqlite")
def execute_query(connection,query): # Da para criar banco de dados aqui e inserir novas linhas
    cursor  = connection.cursor()
    try:
        cursor.execute(query)

        # commit necessario para insert
        connection.commit()

        print('Query executada.')
        if cursor.rowcount != -1:
            print(f"{cursor.rowcount} linha(s) afetadas")
    except Error as e:
        print(f"Erro: '{e}'")

create_item_pedido = """
CREATE TABLE item_pedido (
    id_item INTEGER PRIMARY KEY AUTOINCREMENT,
    id_pedido INTEGER,
    id_produto INTEGER,
    quantidade INTEGER
);"""
execute_query(connection, create_item_pedido)

create_produto_table = """CREATE TABLE produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    tipo VARCHAR(50),
    desc_item VARCHAR(100),
    vl_preco DECIMAL(10,2)
);"""
execute_query(connection,create_produto_table)

#Cria a tabela pedido
create_pedido_table = """CREATE TABLE pedido (
    id_pedido INTEGER PRIMARY KEY AUTOINCREMENT,
    dt_pedido DATE,
    fl_ketchup BOOLEAN,
    desc_uf CHAR(2),
    txt_recado TEXT);"""
execute_query(connection,create_pedido_table)

#Inserindo produto
insert_produto ="""INSERT INTO
produto (tipo,desc_item,vl_preco)
VALUES
('ingrediente', 'camarão', 6),
('massa', 'tradicional', '9.25'),
('borda', 'tradicional', 0),
('queijo', 'muçarela', 4),
('bebida', 'refrigerante', 5);
"""
execute_query(connection,insert_produto)

#Inserindo pedido
insert_pedido = """INSERT INTO
pedido(dt_pedido,fl_ketchup, desc_uf, txt_recado)
VALUES
('2023-06-01', TRUE, 'MG', 'Capricha no queijo!');"""

#inserindo item_pedido
itens = (
    {'id_pedido': 1, 'id_produto': 2, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 3, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 1, 'qtd': 1},
    {'id_pedido': 1, 'id_produto': 4, 'qtd': 2},
    {'id_pedido': 1, 'id_produto': 5, 'qtd': 3}
)
insert_item_pedido ="""INSERT INTO item_pedido (id_pedido, id_produto, quantidade)
VALUES(:id_pedido, :id_produto, :qtd);"""

cursor = connection.cursor()
cursor.executemany(insert_item_pedido,itens)
connection.commit()
cursor.close()


# Funçao de leitura
def execute_read_query(connection,query):
    cursor =connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        
        return result
    except Error as e:
        print(f'Erro: {e}')

#leitura table name

select_table_names = "Select name FROM sqlite_master Where type='table';"
tables = execute_read_query(connection,select_table_names)
print(tables,'\n')

execute_query(connection, "DELETE FROM item_pedido")
execute_query(connection, "DELETE FROM pedido")
execute_query(connection, "DELETE FROM produto")

Connection to SQLite DB successful
Erro: 'table item_pedido already exists'
Erro: 'table produto already exists'
Erro: 'table pedido already exists'
Query executada.
5 linha(s) afetadas
[('item_pedido',), ('sqlite_sequence',), ('produto',), ('pedido',)] 

Query executada.
5 linha(s) afetadas
Query executada.
0 linha(s) afetadas
Query executada.
5 linha(s) afetadas


In [3]:
import pandas as pd
df_pedido = pd.read_csv('pedido.csv')
df_pedido.to_sql ('pedido', connection, if_exists='append', index=False)

count_rows = "SELECT COUNT(id_pedido) as count_id FROM pedido;"
print(execute_read_query(connection, count_rows))

select_all = "SELECT * FROM pedido WHERE id_pedido < 5;"
execute_read_query(connection,select_all)

[(1106,)]


[(0, '2023-05-11', None, 'GO', None),
 (1, '2023-05-11', None, 'PR', 'Aquela pizza perfeita! :-D'),
 (2, '2023-05-11', None, 'SP', 'Muito obrigado!!'),
 (3, '2023-05-11', None, 'SP', None),
 (4, '2023-05-11', None, 'RS', 'Capricha no peperoni')]


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
